# Fixation des Variables

### Transcodage de C a Julia du code du fichier "pqe_bb.c"

Méthodes auxilières

In [3]:
function compute_first_ub(c::Vector{Float64}, Q::Matrix{Float64}, a::Vector{Float64}, b::Float64)
    n = length(c)
    cluster_size = 5
    nb_clusters = div(n, cluster_size)
    UB = 0.0

    for cluster_id in 0:(nb_clusters-1)
        # Id du cluster
        idx = (cluster_id * cluster_size + 1):(cluster_id * cluster_size + cluster_size) # Liste d'indices du cluster donné
        c_cluster = c[idx]
        Q_cluster = Q[idx, idx]
        a_cluster = a[idx]  

        best_val = 0 #-Inf # - Infini mieux au cas ou on a des valeurs négatives

        # Enumérer les 32 combinaisons binaires
        for k in 0:31
            x = reverse(digits(k, base=2, pad=5))  # vector binair
            x_vec = collect(x)                     # convertir le vecteur en Array

            # Vérification des contraintes
            total_weight = 0
            for i in 1:cluster_size
                total_weight = total_weight + a_cluster[i] * x_vec[i]
            end
            if total_weight <= b
                # Evaluer la fonction objectif (linéal + quadratique)
                val = 0
                for i in 1:cluster_size
                    val = val + c_cluster[i] * x_vec[i] # Somme (c_i * x_i)
                end
                 for i in 1:cluster_size-1
                    for j in (i+1):cluster_size
                        val += Q_cluster[i, j] * x_vec[i] * x_vec[j] # Somme (c_ij * x_i * x_j)
                    end
                end

                best_val = max(best_val, val)
            end
        end

        UB += best_val
    end

    return UB
end



compute_first_ub (generic function with 1 method)

In [12]:
struct VarFixInfo
    existe_info::Bool   # une amélioration a été essayé?
    xi_binf::Int        # nouvelle borne inferieur en x_i (0 or 1)
    xi_bsup::Int        # nouvelle borne supérieure en  x_i (0 or 1)
end

In [5]:
function rewrite_problem(
    n::Int, m::Int,
    c::Vector{Float64}, d::Vector{Float64},
    a::Matrix{Float64}, b::Vector{Float64},
    u::Vector{Int},
    varfix::Vector{VarFixInfo},
    I::Vector{Int},
    to0::Bool
)::Float64
    constant = 0.0
    for i in I
        # fixe x_i = v
        v = to0 ? 0 : 1
        varfix[i].existe_info = true
        varfix[i].xi_binf = v
        varfix[i].xi_bsup = v

        # Elimine la variable i du probleme: ajoute sa contribution a la constante
        constant += c[i]*v
        for j in 1:n
            if i != j
                constant += d[min(i,j), max(i,j)] * v * (u[j] > 0 ? 1 : 0)
            end
        end

        u[i] = 0
        for j in 1:m
            b[j] -= a[j,i]*v
            a[j,i] = 0.0
        end
        c[i] = 0.0
        for j in 1:n
            d[min(i,j), max(i,j)] = 0.0
        end
    end
    return constant
end

rewrite_problem (generic function with 1 method)

In [6]:
"""
    calcul_borne2(c,Q,a,b) -> ub

Compute the Lagrangian bound2 for the full problem via compute_first_ub.
"""
function calcul_borne2(
    c::Vector{Float64},
    Q::Matrix{Float64},
    a::Vector{Float64},
    b::Float64
)::Float64
    return compute_first_ub(c, Q, a, b)
end

calcul_borne2

In [7]:
"""
    calcul_borne2_I(c,Q,a,b,I) -> ub

Rewrite x[I]=0, then call calcul_borne2 on the reduced system and
add back the constant term.
"""
function calcul_borne2_I(
    c::Vector{Float64},
    Q::Matrix{Float64},
    a::Vector{Float64},
    b::Float64,
    I::Vector{Int}
)::Float64
    # fixe to 0
    c2, Q2, a2, b2, const0 = rewrite_problem(c, Q, a, b, I, 0)
    ub2 = calcul_borne2(c2, Q2, a2, b2)
    return ub2 + const0
end

calcul_borne2_I

Fixations des variables a 0

In [8]:
"""
    fix_variables_0(n, m, c, d, a, b, u, x_inf, bound_inf, dual_mul, varfix;
                    packet_size=2)

Scans variables with u[i]==1 ∧ x_inf[i]==0 in packets of `packet_size`.
For each packet I:
  - compute bound2_I;
  - if bound2_I < bound_inf, fix all i∈I to 0 in `varfix`.
"""
function fix_variables_0(
    n::Int, m::Int,
    c::Vector{Float64}, d::Matrix{Float64},
    a::Matrix{Float64}, b::Vector{Float64},
    u::Vector{Int}, x_inf::Vector{Float64},
    bound_inf::Float64,
    dual_mul::Vector{Float64},
    varfix::Vector{VarFixInfo};
    packet_size::Int=2
)
    I = Int[]
    i = 1
    while true
        empty(I)
        while i ≤ n && length(I) < packet_size
            if u[i]==1 && abs(x_inf[i]) < 1e-9
                push(I, i)
            end
            i += 1
        end
        isempty(I) && break

        b2 = Ref(0.0)
        y_tmp = zeros(Float64, m)
        calcul_borne2_I(n,m,c,d,a,b,u,dual_mul,I,b2,y_tmp)
        if b2[] < bound_inf
            for ii in I
                varfix[ii].existe_info = true
                varfix[ii].xi_binf = 0
                varfix[ii].xi_bsup = 0
            end
        end
    end
end

fix_variables_0

Fixations des variables a 1

In [11]:
"""
    fix_variables_01(n,m,c,d,a,b,u,x_inf,bound_inf,dual_mul,varfix)

For each i with u[i]==1 ∧ not yet fixed:
  - try fix to 0: bound2_{i} < bound_inf ⇒ confirm x_i=0.
  - else try fix to 1 similarly.
"""
function fix_variables_01(
    n::Int, m::Int,
    c::Vector{Float64}, d::Matrix{Float64},
    a::Matrix{Float64}, b::Vector{Float64},
    u::Vector{Int}, x_inf::Vector{Float64},
    bound_inf::Float64,
    dual_mul::Vector{Float64},
    varfix::Vector{VarFixInfo}
)
    for i in 1:n
        if u[i]==1 && !varfix[i].existe_info
            # essaye x_i=0
            b2 = Ref(0.0); y_tmp = zeros(Float64, m)
            calcul_borne2_I(n,m,c,d,a,b,u,dual_mul,[i],b2,y_tmp)
            if b2[] < bound_inf
                varfix[i].existe_info = true
                varfix[i].xi_binf = 0
                varfix[i].xi_bsup = 0
                continue
            end
            # essaye x_i=1
            # réecrit le sous-problème en forcant i→1:
            const_term = rewrite_problem(n,m,c,d,a,b,u,varfix,[i],false)
            calcul_borne2(n,m,c,d,a,b,u,dual_mul,b2,y_tmp)
            b2[] += const_term
            if b2[] < bound_inf
                varfix[i].existe_info = true
                varfix[i].xi_binf = 1
                varfix[i].xi_bsup = 1
                continue
            end
            # Si les deux rattent, on fixe pas
            
        end
    end
end

fix_variables_01